In [15]:
from dask_cuda import LocalCUDACluster
from dask.distributed import Client
import dask_cudf
import cudf
# Create a Dask single-node CUDA cluster w/ one worker per device
# cluster = LocalCUDACluster(protocol="ucx",
#                            enable_tcp_over_ucx=True,                           
#                            enable_infiniband=False)

# client = Client(cluster)

# Read CSV file in parallel across workers

gdf = cudf.read_csv("airline_2m.csv")


In [16]:
to_drop = [
    'Month',
    'DayofMonth', 
    'Reporting_Airline',        # Airline Unique Carrier Code
    'DOT_ID_Reporting_Airline', # Number assigned by US DOT to identify a unique airline
    'IATA_CODE_Reporting_Airline', #Airline Code assigned by IATA
    'Tail_Number', # Aircraft Tailn Number
    'Flight_Number_Reporting_Airline', # Flight Number
    'OriginAirportID', # Origin Airport ID
    'OriginAirportSeqID', # Origin Airport Sequence ID
    'OriginCityMarketID', # Origin City Market ID
    'OriginStateFips', # Origin State FIPS Place Code
    'OriginStateName', # Origin State Name, Origin State is stored in OriginState
    'OriginWac', # World area code for origin airport
    
     # Stored in Dest, DestCityName and DestState
    'DestAirportID',
    'DestAirportSeqID', 
    'DestCityMarketID',
    'DestCityName',
    'DestStateFips',
    'DestStateName',
    'DestWac',
    # DivActualElapsesTime captures length of departure 
    # and DivReachedDest shows whether flight actually reached
    'DivDistance',
    'Div1Airport',
    'Div1AirportID',
    'Div1AirportSeqID',
    'Div1WheelsOn',
    'Div1TotalGTime',
    'Div1LongestGTime',
    'Div1WheelsOff',
    'Div1TailNum',
    'Div2Airport',
    'Div2AirportID',
    'Div2AirportSeqID',
    'Div2WheelsOn',
    'Div2TotalGTime',
    'Div2LongestGTime',
    'Div2WheelsOff',
    'Div2TailNum',
    'Div3Airport',
    'Div3AirportID',
    'Div3AirportSeqID',
    'Div3WheelsOn',
    'Div3TotalGTime',
    'Div3LongestGTime',
    'Div3WheelsOff',
    'Div3TailNum',
    'Div4Airport',
    'Div4AirportID',
    'Div4AirportSeqID',
    'Div4WheelsOn',
    'Div4TotalGTime',
    'Div4LongestGTime',
    'Div4WheelsOff',
    'Div4TailNum',
    'Div5Airport',
    'Div5AirportID',
    'Div5AirportSeqID',
    'Div5WheelsOn',
    'Div5TotalGTime',
    'Div5LongestGTime',
    'Div5WheelsOff',
    'Div5TailNum'
]

gdf = gdf.drop(to_drop, axis=1)
gdf.columns

Index(['Year', 'Quarter', 'DayOfWeek', 'FlightDate', 'Origin',
       'OriginCityName', 'OriginState', 'Dest', 'DestState', 'CRSDepTime',
       'DepTime', 'DepDelay', 'DepDelayMinutes', 'DepDel15',
       'DepartureDelayGroups', 'DepTimeBlk', 'TaxiOut', 'WheelsOff',
       'WheelsOn', 'TaxiIn', 'CRSArrTime', 'ArrTime', 'ArrDelay',
       'ArrDelayMinutes', 'ArrDel15', 'ArrivalDelayGroups', 'ArrTimeBlk',
       'Cancelled', 'CancellationCode', 'Diverted', 'CRSElapsedTime',
       'ActualElapsedTime', 'AirTime', 'Flights', 'Distance', 'DistanceGroup',
       'CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay',
       'LateAircraftDelay', 'FirstDepTime', 'TotalAddGTime', 'LongestAddGTime',
       'DivAirportLandings', 'DivReachedDest', 'DivActualElapsedTime',
       'DivArrDelay'],
      dtype='object')

In [17]:
gdf.head()

,Year,Quarter,DayOfWeek,FlightDate,Origin,OriginCityName,OriginState,Dest,DestState,CRSDepTime,...,NASDelay,SecurityDelay,LateAircraftDelay,FirstDepTime,TotalAddGTime,LongestAddGTime,DivAirportLandings,DivReachedDest,DivActualElapsedTime,DivArrDelay
0,1998,1,5,1998-01-02,MSP,"Minneapolis, MN",MN,SLC,UT,1640,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,2009,2,4,2009-05-28,MKE,"Milwaukee, WI",WI,MCO,FL,1204,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0.0,<NA>,<NA>,<NA>
2,2013,2,6,2013-06-29,GJT,"Grand Junction, CO",CO,DFW,TX,1630,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0.0,<NA>,<NA>,<NA>
3,2010,3,2,2010-08-31,LAX,"Los Angeles, CA",CA,DTW,MI,1305,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0.0,<NA>,<NA>,<NA>
4,2006,1,7,2006-01-15,EWR,"Newark, NJ",NJ,CLT,NC,1820,...,0.0,0.0,32.0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [20]:
gdf.isna().sum()

Year                          0
Quarter                       0
DayOfWeek                     0
FlightDate                    0
Origin                        0
OriginCityName                0
OriginState                 646
Dest                          0
DestState                   594
CRSDepTime                    0
DepTime                   36005
DepDelay                  36068
DepDelayMinutes           36068
DepDel15                  36068
DepartureDelayGroups      36068
DepTimeBlk                    0
TaxiOut                  415642
WheelsOff                415677
WheelsOn                 417958
TaxiIn                   417847
CRSArrTime                    0
ArrTime                   39551
ArrDelay                  41078
ArrDelayMinutes           41078
ArrDel15                  41078
ArrivalDelayGroups        41078
ArrTimeBlk                    0
Cancelled                     0
CancellationCode        1979780
Diverted                      0
CRSElapsedTime              281
ActualEl

In [21]:
!nvidia-smi

Thu May 27 16:17:25 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.80       Driver Version: 460.80       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce GTX 106...  Off  | 00000000:01:00.0 Off |                  N/A |
| N/A   62C    P5     8W /  N/A |   1474MiB /  6078MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [23]:
gdf.to_csv('selected.csv', index=False)